In [ ]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('assets/visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('assets/cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('assets/checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('assets/purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [4]:
visits.head(5)

,user_id,visit_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00


In [5]:
cart.head(5)

,user_id,cart_time
0,2be90e7c-9cca-44e0-bcc5-124b945ff168,2017-11-07 20:45:00
1,4397f73f-1da3-4ab3-91af-762792e25973,2017-05-27 01:35:00
2,a9db3d4b-0a0a-4398-a55a-ebb2c7adf663,2017-03-04 10:38:00
3,b594862a-36c5-47d5-b818-6e9512b939b3,2017-09-27 08:22:00
4,a68a16e2-94f0-4ce8-8ce3-784af0bbb974,2017-07-26 15:48:00


In [6]:
checkout.head(5)

,user_id,checkout_time
0,d33bdc47-4afa-45bc-b4e4-dbe948e34c0d,2017-06-25 09:29:00
1,4ac186f0-9954-4fea-8a27-c081e428e34e,2017-04-07 20:11:00
2,3c9c78a7-124a-4b77-8d2e-e1926e011e7d,2017-07-13 11:38:00
3,89fe330a-8966-4756-8f7c-3bdbcd47279a,2017-04-20 16:15:00
4,3ccdaf69-2d30-40de-b083-51372881aedd,2017-01-08 20:52:00


In [7]:
purchase.head(5)

,user_id,purchase_time
0,4b44ace4-2721-47a0-b24b-15fbfa2abf85,2017-05-11 04:25:00
1,02e684ae-a448-408f-a9ff-dcb4a5c99aac,2017-09-05 08:45:00
2,4b4bc391-749e-4b90-ab8f-4f6e3c84d6dc,2017-11-20 20:49:00
3,a5dbb25f-3c36-4103-9030-9f7c6241cd8d,2017-01-22 15:18:00
4,46a3186d-7f5a-4ab9-87af-84d05bfd4867,2017-06-11 11:32:00


Step 2: Left merging visits and cart

In [8]:
visits_cart = pd.merge(visits, cart, how="left")

Step 3: How long is `visits_cart`?

In [9]:
len(visits_cart)

2000

Step 4: How many timestamps are null for `cart_time`?

In [15]:
cn = len(visits_cart[visits_cart['cart_time'].isna()])

Step 5: What percentage only visited?

In [18]:
vn = len(visits_cart)
percentage_visited_only = ((cn / vn) * 100)
percentage_visited_only

82.6

Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [19]:
cart_checkout = pd.merge(cart, checkout, how="left")
percentage_cart_only =  (len(cart_checkout[cart_checkout['checkout_time'].isna()]) / len(cart_checkout)) * 100
percentage_cart_only

35.05747126436782

Step 7: Merge it all together

In [20]:
all_data = visits_cart.merge(cart_checkout, how="left").merge(purchase, how="left")
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT


Step 8: % of users who got to checkout but did not purchase

In [25]:
checkout_purchase = checkout.merge(purchase, how="left")
percentage_checkout_only = (len(checkout_purchase[checkout_purchase['purchase_time'].isna()]) / len(checkout_purchase)) * 100
percentage_checkout_only

24.550898203592812

Step 9: check each part of the funnel, let's print all 3 of them again

In [22]:
print(percentage_visited_only)
print(percentage_cart_only)
print(percentage_checkout_only)

82.6
35.05747126436782
24.550898203592812


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [27]:
all_data['time_to_purchase'] = all_data.purchase_time - all_data.visit_time

In [30]:
all_data.head()

,user_id,visit_time,cart_time,checkout_time,purchase_time,time_to_purchase
0,943647ef-3682-4750-a2e1-918ba6f16188,2017-04-07 15:14:00,NaT,NaT,NaT,NaT
1,0c3a3dd0-fb64-4eac-bf84-ba069ce409f2,2017-01-26 14:24:00,2017-01-26 14:44:00,2017-01-26 14:54:00,2017-01-26 15:08:00,0 days 00:44:00
2,6e0b2d60-4027-4d9a-babd-0e7d40859fb1,2017-08-20 08:23:00,2017-08-20 08:31:00,NaT,NaT,NaT
3,6879527e-c5a6-4d14-b2da-50b85212b0ab,2017-11-04 18:15:00,NaT,NaT,NaT,NaT
4,a84327ff-5daa-4ba1-b789-d5b4caf81e96,2017-02-27 11:25:00,NaT,NaT,NaT,NaT


In [31]:
# Average time to purchase
all_data.time_to_purchase.mean()

Timedelta('0 days 00:43:12.380952380')